In [35]:
# import pandas as pd
# import os
# import re
# import json
# import yaml
# from glob import glob
# from tqdm import tqdm
# from pprint import pprint
# import torch
# import pytorch_lightning as pl
# from rouge import Rouge # 모델의 성능을 평가하기 위한 라이브러리입니다.

# from torch.utils.data import Dataset , DataLoader
# from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
# from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
# from transformers import Trainer, TrainingArguments
# from transformers import EarlyStoppingCallback
# from peft import get_peft_config, get_peft_model, LoraConfig, IA3Config, LoKrConfig, LoHaConfig, AdaLoraConfig, TaskType

# import wandb # 모델 학습 과정을 손쉽게 Tracking하고, 시각화할 수 있는 라이브러리입니다.

In [41]:
# 트랜스포머 라이브러리에서 사용할 클래스, 기능 가져오기
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling

# 데이터셋 라이브러리
from datasets import load_dataset

# peft 라이브러리
from peft import get_peft_model, PromptTuningConfig, TaskType, PromptTuningInit, PeftModel, PeftConfig

import os
import time

In [42]:
model_name = "EleutherAI/polyglot-ko-1.3b"

In [43]:
#사전 학습 모델에 사용할 tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

#지정한 모델 이름 사용해 사전 학습된 casual LM 로드
model = AutoModelForCausalLM.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [45]:
# dataset 라이브러리 내 load_dataset 기능 활용해 "english_quotes" 데이터셋 가져오기
dataset = load_dataset('csv', data_files={'train': "../data/train.csv"})
# 지정된 토크나이저 활용해 데이터셋 명언 토큰화
data = dataset.map(lambda samples:tokenizer(samples["dialogue"]), batched = True)

# 훈련 샘플 일부 선택(처음 50개)
train_sample = data["train"].select(range(50))

# train_sample 출력해보기
display(train_sample) 

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/12457 [00:00<?, ? examples/s]

Dataset({
    features: ['fname', 'dialogue', 'summary', 'topic', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 50
})

In [39]:
# 지정된 토크나이저 사용해 입력 텍스트 토큰화
input1 = tokenizer("Two things are infinite:", return_tensors = "pt", padding = True)

# input_ids와 attention_mask 기반해 사전 학습된 파운데이션 모델 사용해 텍스트 생성 요청
foundation_outputs = model.generate(
 input_ids = input1["input_ids"],
    attention_mask = input1["attention_mask"],
    max_new_tokens = 7,
    eos_token_id = tokenizer.eos_token_id
    )

# 생성된 token id를 사람이 읽을 수 있는 텍스트로 디코딩
decoded_output = tokenizer.batch_decode(foundation_outputs, skip_special_tokens = True)

# 디코딩된 텍스트 출력(생성된 결과물 사람이 읽을 수 있는 형태로 출력)
print(decoded_output)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['Two things are infinite: the first is that']


In [ ]:
# dataset 라이브러리 내 load_dataset 기능 활용해 "english_quotes" 데이터셋 가져오기
data = load_dataset("Abirate/english_quotes")

# 지정된 토크나이저 활용해 데이터셋 명언 토큰화
data = data.map(lambda samples:tokenizer(samples["quote"]), batched = True)

# 훈련 샘플 일부 선택(처음 50개)
train_sample = data["train"].select(range(50))

# train_sample 출력해보기
display(train_sample)